In [0]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

from collections import Counter
from matplotlib import pyplot as plt
% matplotlib inline

In [0]:
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


def Fermi(E,mu):
    b=40
    fermival=1 / (1 + np.exp((E - mu) * b))
    return fermival

def Transmission(E,E0,Gamma1,Gamma2,V):
    SumGammaSq=(Gamma1 + Gamma2)**2
    E0split=(E - (E0 + ((Gamma1 - Gamma2) / (Gamma1 + Gamma2)) * V/2))**2
    Transmissionval=(4* Gamma1*Gamma2)/(E0split + SumGammaSq)
    return Transmissionval

def Integrand(x,E0,G1,G2,V):
    Integrand_val= Transmission(x,E0,G1,G2,V)*(Fermi(x,V/2)-Fermi(x,-V/2))
    return Integrand_val

def Integrate(E0,G1,G2,V):
    y=lambda x:Integrand(x,E0,G1,G2,V)
    result=integrate.quad(y, -2, 2)
    return result
    

def Current(E0,G1,G2,V):
    I=Integrate(E0,G1,G2,V)
    return I[0]*float(7.74809174e-5)

In [0]:
def residual(params, bias, current):
  return current - Current(params[0], params[1], params[2], bias)

def dTddE(E0, G1, G2, V, E):
  SumGammaSq=(G1 + G2)**2
  E0split=(E - (E0 + ((G1 - G2) / (G1 + G2)) * V/2))
  #Transmissionval=(4* Gamma1*Gamma2)/(E0split + SumGammaSq)
  derivative = -(4* G1*G2)/((E0split**2 + SumGammaSq)**2)*2*E0split
  return derivative

def ddEdE0(E0, G1, G2, V, E):
  return -1

def dTdG1(E0, G1, G2, V, E):
  SumGammaSq = (G1 + G2)**2
  E0split = (E-(E0 + ((G1 - G2)/(G1 + G2))*V/2))**2
  derivative = 4*G2/(E0split + SumGammaSq)
  return derivative

def dTdG2(E0, G1, G2, V, E):
  SumGammaSq = (G1 + G2)**2
  E0split = (E-(E0 + ((G1 - G2)/(G1 + G2))*V/2))**2
  derivative = 4*G1/(E0split + SumGammaSq)
  return derivative

def dTdG(E0, G1, G2, V, E):
  SumGamma = G1 + G2
  E0split = (E-(E0 + ((G1 - G2)/SumGamma)*V/2))**2
  derivative = -4*G1*G2/(E0split + SumGamma**2)**2 * 2*SumGamma
  return derivative

def dGdG1(E0, G1, G2, V, E):
  return 1

def dGdG2(E0, G1, G2, V, E):
  return 1

def ddEdG1(E0, G1, G2, V, E):
  SumGamma = G1 + G2
  return -G2*V/(SumGamma**2)

def ddEdG2(E0, G1, G2, V ,E):
  SumGamma = G1 + G2
  return G1*V/(SumGamma**2)

def current_gradient(params, bias, current):
  full_dTdE0 = lambda x: dTddE(params[0], params[1], params[2], bias, x) * ddEdE0(params[0], params[1], params[2], bias, x)
  full_dTdG1 = lambda x: (dTddE(params[0], params[1], params[2], bias, x) * ddEdG1(params[0], params[1], params[2], bias, x) +
                          dTdG1(params[0], params[1], params[2], bias, x) +
                          dTdG(params[0], params[1], params[2], bias, x)*dGdG1(params[0], params[1], params[2], bias, x))
  full_dTdG2 = lambda x: (dTddE(params[0], params[1], params[2], bias, x) * ddEdG2(params[0], params[1], params[2], bias, x) +
                          dTdG2(params[0], params[1], params[2], bias, x) +
                          dTdG(params[0], params[1], params[2], bias, x)*dGdG2(params[0], params[1], params[2], bias, x))
  full_dIdE0 = integrate.quad(lambda x: full_dTdE0(x)*(Fermi(x, bias/2)-Fermi(x, -bias/2)), -2, 2)[0] * float(7.74809174e-5)
  full_dIdG1 = integrate.quad(lambda x: full_dTdG1(x)*(Fermi(x, bias/2)-Fermi(x, -bias/2)), -2, 2)[0] * float(7.74809174e-5)
  full_dIdG2 = integrate.quad(lambda x: full_dTdG2(x)*(Fermi(x, bias/2)-Fermi(x, -bias/2)), -2, 2)[0] * float(7.74809174e-5)
  return full_dIdE0, full_dIdG1, full_dIdG2

                         

In [0]:
class TransmissionCalculator():
  def __init__(V):
    self.V = V
    self.Add = 0
    self.SumGamma = 0
    self.E0split = 0
    self.denominator = 0
    
  def _forward(self, E0, G1, G2):
    self.E0 = E0
    self.G1 = G1
    self.G2 = G2
    self.SumGamma = G1 + G2
    self.prod4 = 4*G1*G2
    self.SumGammaSq = self.SumGamma**2
    self.Add = E0 + ((G1 - G2)/SumGamma)*V/2
    return (lambda x: self.prod4/((x-self.Add)**2 + self.SumGammaSq))
  
  def _backward(self):
    dTddE = lambda x: -(self.prod4/((x-self.Add)**2 + self.SumGammaSq)**2) * 2 * (x - self.Add)
    dTdG = lambda x: 4*self.G1*self.G2/ ((x - self.Add)**2 + self.SumGammaSq)**2 * 2 * self.SumGamma
    dTdG1 = lambda x: 4*self.G2 / ((x - self.Add)**2 + self.SumGammaSq) + dTdG(x)
    dTdG2 = lambda x: 4*self.G1 / ((x - self.Add)) + dTdG(x)
    return dTddE, dTdG1, dTdG2
  
  def get_forward(self):
    return np.vectorize(self._forward)
  

In [0]:
class CurrentCalculator():
  def __init__(V):
    self.V = V
    self.T = TransmissionCalculator(V)
    
  def _forward(self, E0, G1, G2):
    T_val = self.T.forward(E0, G1, G2)
    Integrand_val = lambda x: T_val(x)*(Fermi(x,V/2)-Fermi(x,-V/2))
    return integrate.quad(Integrand_val, -2, 2)
              
  def _backward(self):
    dTddE, dTdG1, dTdG2 = self.T.backward()
    full_dE0 = lambda x: -dTddE(x)*(Fermi(x,V/2)-Fermi(x,-V/2))
    add_mult = dTddE*self.V/self.T.SumGammaSq
    full_dG1 = lambda x: (dTdG1 - add_mult * self.T.G2)*(Fermi(x,V/2)-Fermi(x,-V/2))
    full_dG2 = lambda x: (dTdG2 + add_mult * self.T.G1)*(Fermi(x,V/2)-Fermi(x,-V/2))
    return integrate.quad(full_dE0, -2, 2), integrate.quad(full_dG1, -2, 2), integrate.quad(full_dG2, -2, 2)
                                                        
    

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/molecular electronics'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/molecular electronics


In [0]:
PATH_TO_FILE = 'C60/000001.txt'

In [0]:
data = pd.read_csv(PATH_TO_FILE, comment = '#', header = None, delimiter='\t')

In [0]:
data[400:1200][0].head()

400    2.000060
401    1.995072
402    1.990015
403    1.985029
404    1.980106
Name: 0, dtype: float64

In [0]:
vec_residual = np.vectorize(residual, excluded = [0])

In [0]:
%time result = least_squares(fun=np.vectorize(residual, excluded = [0]), jac=lambda params, bias, current: np.transpose(np.vectorize(current_gradient, excluded = [0])(params, bias, current)),  x0=[0.5, 5e-3, 5e-3], ftol = 1e-12, gtol = 1e-12, xtol = 1e-12, method = 'lm', args = (data[400:1200][0],data[400:1200][1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: IntegrationWarning: The integral is probably divergent, or slowly convergent.


ValueError: ignored

In [0]:
import pickle

In [0]:
with open('results.pickle') as f:
  pickle.dump(f, result)

In [0]:
def gradient_calculation(E, )